In [1]:
# -*- coding: UTF-8 -*-
from __future__ import unicode_literals, division, print_function, absolute_import
from builtins import range, str, zip
import re
import os
from glob import glob
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from nlputils.features import preprocess_text
from nlputils.embedding import proj2d
from nlputils.visualize import basic_viz

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Extract emotions and sentiment from comments

In [2]:
# read in sentiment words
df = pd.read_csv("GermanPolarityClues-2012/GermanPolarityClues-Negative-21042012.tsv", sep="\t", header=None)
sent_neg = set(preprocess_text(t) for t in df[0])
df = pd.read_csv("GermanPolarityClues-2012/GermanPolarityClues-Positive-21042012.tsv", sep="\t", header=None)
sent_pos = set(preprocess_text(t) for t in df[0])
df = pd.read_csv("GermanPolarityClues-2012/GermanPolarityClues-Neutral-21042012.tsv", sep="\t", header=None)
sent_neu = set(preprocess_text(t) for t in df[0])
# read in emotion words and assign to 4 emotions
# https://www.microsoft.com/developerblog/2015/11/29/emotion-detection-and-recognition-from-text-using-deep-learning/
df = pd.read_csv("emotion_words/Ekel.txt", sep="\t", header=None)
emo_anger = set(preprocess_text(t) for t in df[0])
df = pd.read_csv("emotion_words/Verachtung.txt", sep="\t", header=None)
emo_anger.update(set(preprocess_text(t) for t in df[0]))
df = pd.read_csv("emotion_words/Wut.txt", sep="\t", header=None)
emo_anger.update(set(preprocess_text(t) for t in df[0]))
df = pd.read_csv("emotion_words/Furcht.txt", sep="\t", header=None)
emo_fear = set(preprocess_text(t) for t in df[0])
df = pd.read_csv("emotion_words/Freude.txt", sep="\t", header=None)
emo_happy = set(preprocess_text(t) for t in df[0])
df = pd.read_csv("emotion_words/Ueberraschung.txt", sep="\t", header=None)
emo_happy.update(set(preprocess_text(t) for t in df[0]))
df = pd.read_csv("emotion_words/Trauer.txt", sep="\t", header=None)
emo_sad = set(preprocess_text(t) for t in df[0])
# define emotion emojis
# https://www.insight-centre.org/sites/default/files/publications/ianwood-emotionworkshop.pdf
emojis_happy = set("😀😃😄😁😆😅😂🤣☺😊😇🙂🙃😉😌😍😘😗😙😚😋😛😝😜🤪🤩😎😺😸😹😻😽🙌👏👍✌👌🎉❤💕💞💓💗💖♥")
emojis_sad = set("😞😔😟😕🙁☹😣😖😫😩😢😭😥😶😐😧😦😪😿🙀👎💔")
emojis_anger = set("😒😑😤😠😡🤬😾🖕💩👿")
emojis_fear = set("😱😳😨😰😵😧😮😲🙀")
emotions_list = ["happy", "sad", "fear", "anger", "-"]

def label_word_sent(w):
    sent_label = 0.
    if w in sent_neg:
        sent_label = -1.
    elif w in sent_pos:
        sent_label = +1.
    return sent_label

def label_word_emo(w):
    emo_label = "-"
    if w in emo_anger:
        emo_label = "anger"
    elif w in emo_fear:
        emo_label = "fear"
    elif w in emo_happy:
        emo_label = "happy"
    elif w in emo_sad:
        emo_label = "sad"
    return emo_label
    

In [21]:
comments = {}
comments_pp = {}
sent_scores = {}
emo_anger_scores = {}
emo_fear_scores = {}
emo_sad_scores = {}
emo_happy_scores = {}
emo_main_label = {}
fnames = sorted(glob("Kommentare/*.txt"))
for fname in fnames:
    print("##########")
    print(fname)
    comments[fname] = {}
    comments_pp[fname] = {}
    sent_scores[fname] = {}
    emo_anger_scores[fname] = {}
    emo_fear_scores[fname] = {}
    emo_sad_scores[fname] = {}
    emo_happy_scores[fname] = {}
    emo_main_label[fname] = {}
    with open(fname) as f:
        for i, l in enumerate(f.readlines()):
            if l.strip():
                pp_l = preprocess_text(l)
                if pp_l:
                    comments[fname][i] = l
                    # based on the preprocessed words, get sentiment and word based emotion scores
                    pp_l = preprocess_text(l)
                    sent_scores[fname][i] = np.mean([label_word_sent(word) for word in pp_l.split()])
                    emotions = Counter([label_word_emo(word) for word in pp_l.split()])
                    # add scores based on emojis (from non-preprocessed text!)
                    char_counts = Counter(list(l))
                    for c in char_counts:
                        if len(c.encode("ascii", "backslashreplace")) > 5:
                            pp_l += " " + " ".join(char_counts[c]*[c])
                            if c in emojis_anger:
                                emotions["anger"] += char_counts[c]
                            elif c in emojis_fear:
                                emotions["fear"] += char_counts[c]
                            elif c in emojis_happy:
                                emotions["happy"] += char_counts[c]
                            elif c in emojis_sad:
                                emotions["sad"] += char_counts[c]
                    comments_pp[fname][i] = pp_l
                    n_tokens = len(pp_l.split())
                    emotions["-"] = 0
                    emo_main_label[fname][i] = max(emotions, key=emotions.get)
                    emo_anger_scores[fname][i] = emotions["anger"]/n_tokens
                    emo_fear_scores[fname][i] = emotions["fear"]/n_tokens
                    emo_sad_scores[fname][i] = emotions["sad"]/n_tokens
                    emo_happy_scores[fname][i] = emotions["happy"]/n_tokens
    print("sentiment scores: [%.3f, %.3f]; %.3f +/- %.3f" % (max(sent_scores[fname].values()), min(sent_scores[fname].values()), np.mean(list(sent_scores[fname].values())), np.std(list(sent_scores[fname].values()))))
    i = max(sorted(comments[fname], key=sent_scores[fname].get, reverse=True)[:10], key=lambda x: len(comments[fname][x]))
    print("most positive comment (%.3f):" % sent_scores[fname][i])
    print(comments[fname][i])
    i = max(sorted(comments[fname], key=sent_scores[fname].get)[:10], key=lambda x: len(comments[fname][x]))
    print("most negative comment (%.3f):" % sent_scores[fname][i])
    print(comments[fname][i])
    print("anger scores: [%.3f, %.3f]; %.3f +/- %.3f" % (max(emo_anger_scores[fname].values()), min(emo_anger_scores[fname].values()), np.mean(list(emo_anger_scores[fname].values())), np.std(list(emo_anger_scores[fname].values()))))
    i = max(sorted(comments[fname], key=emo_anger_scores[fname].get, reverse=True)[:10], key=lambda x: len(comments[fname][x]))
    print("most angry comment (%.3f):" % emo_anger_scores[fname][i])
    print(comments[fname][i])
    i = max(sorted(comments[fname], key=emo_fear_scores[fname].get, reverse=True)[:10], key=lambda x: len(comments[fname][x]))
    print("fear scores: [%.3f, %.3f]; %.3f +/- %.3f" % (max(emo_fear_scores[fname].values()), min(emo_fear_scores[fname].values()), np.mean(list(emo_fear_scores[fname].values())), np.std(list(emo_fear_scores[fname].values()))))
    print("most afraid comment (%.3f):" % emo_fear_scores[fname][i])
    print(comments[fname][i])
    i = max(sorted(comments[fname], key=emo_sad_scores[fname].get, reverse=True)[:10], key=lambda x: len(comments[fname][x]))
    print("sad scores: [%.3f, %.3f]; %.3f +/- %.3f" % (max(emo_sad_scores[fname].values()), min(emo_sad_scores[fname].values()), np.mean(list(emo_sad_scores[fname].values())), np.std(list(emo_sad_scores[fname].values()))))
    print("most sad comment (%.3f):" % emo_sad_scores[fname][i])
    print(comments[fname][i])
    i = max(sorted(comments[fname], key=emo_happy_scores[fname].get, reverse=True)[:10], key=lambda x: len(comments[fname][x]))
    print("happy scores: [%.3f, %.3f]; %.3f +/- %.3f" % (max(emo_happy_scores[fname].values()), min(emo_happy_scores[fname].values()), np.mean(list(emo_happy_scores[fname].values())), np.std(list(emo_happy_scores[fname].values()))))
    print("most happy comment (%.3f):" % emo_happy_scores[fname][i])
    print(comments[fname][i])

##########
Kommentare/001 eishockey.txt
sentiment scores: [1.000, -1.000]; 0.138 +/- 0.306
most positive comment (1.000):
Hammer Leistung 👍👍😝Herzlichen Glückwunsch 🎈🎉

most negative comment (-0.600):
Mario Bratfisch schlechter Verlierer? 🤔 Frust?

anger scores: [0.600, 0.000]; 0.012 +/- 0.046
most angry comment (0.471):
Des drecks Russenpack ist doch voll bis oben hin🤬🤬🤬🤬🤬😠😠😠

fear scores: [0.100, 0.000]; 0.001 +/- 0.007
most afraid comment (0.028):
Bin ja schon lange nicht mehr  nachts wegen einer TV übertragung aufgestenden. Früher war es die Mondlandung, Cassius Clay oder Schumi. Ich muss aber sagen Respekt vor den Eishockey Cracks hat sich gelohnt den tollen Kampf um Gold anzuschauen. Wir waren so dicht dran.😧 Was ich auch  super finde, sind die Trikots der Deutschen Eishockey  Nationalmannschaft, Schwarz Rot Gold. Sieht wirklich klasse aus, nicht so wie die Fussballer mit ihrer Altkleidersammlung.

sad scores: [0.500, 0.000]; 0.004 +/- 0.025
most sad comment (0.125):
So geil haben

sentiment scores: [1.000, -1.000]; -0.021 +/- 0.177
most positive comment (1.000):
Vollkommen richtig!

most negative comment (-1.000):
Nicht schlecht

anger scores: [1.000, 0.000]; 0.034 +/- 0.099
most angry comment (1.000):
Dumpfbacke....

fear scores: [0.500, 0.000]; 0.002 +/- 0.016
most afraid comment (0.200):
Ob, H4, Flüchtling, Obdachlosigkeit usw

sad scores: [0.500, 0.000]; 0.005 +/- 0.027
most sad comment (0.308):
Der Vollposten soll mal Hartz 4 bekommen, und damit leben,bevor er so eine scheisse redet. Armes Deutschland 👎👎👎👎👎👎👎👎

happy scores: [1.000, 0.000]; 0.022 +/- 0.067
most happy comment (0.625):
Ne sind stinke reich😂😂😂😂

##########
Kommentare/012 hawking tot.txt
sentiment scores: [1.000, -1.000]; 0.079 +/- 0.269
most positive comment (0.667):
Sehr schöne vorstellung....

most negative comment (-0.500):
Was ist dein Problem?

anger scores: [0.500, 0.000]; 0.005 +/- 0.031
most angry comment (0.115):
Patrick Fin. Schwachsinn. Die Menschheit wird nicht dümmer, nur weil ein

sentiment scores: [1.000, -1.000]; 0.039 +/- 0.219
most positive comment (1.000):
Ja schon 😂😂

most negative comment (-0.571):
OMG fast 2 Kilometer was ne Strafe....

anger scores: [0.333, 0.000]; 0.012 +/- 0.031
most angry comment (0.200):
Na und??? Soweit War eh mein Schulweg täglich 😡😡

fear scores: [0.333, 0.000]; 0.002 +/- 0.014
most afraid comment (0.091):
So schlimm ist das nun wirklich nicht...  Ich bin auch täglich in die schule gegangen - zu Fuß bei jedem Wetter 😱😱😱 und die schule war auch nicht gleich ums Eck

sad scores: [0.500, 0.000]; 0.003 +/- 0.022
most sad comment (0.125):
Richtiig so ... mobbing ist die Krankheit unserer Gesellschaft

happy scores: [1.000, 0.000]; 0.072 +/- 0.144
most happy comment (1.000):
😆Prima !!!!

##########
Kommentare/023 essener tafel.txt
sentiment scores: [1.000, -1.000]; 0.074 +/- 0.255
most positive comment (1.000):
Sehr guter Beitrag

most negative comment (-0.429):
Schmarotzer und unsere Trümmerfrauen gehen leer aus

anger scores: [1.000,

sentiment scores: [1.000, -0.750]; 0.090 +/- 0.274
most positive comment (1.000):
Super 👏🏻👏🏻👏🏻👏🏻👏🏻👍🏻👍🏻👍🏻👍🏻

most negative comment (-0.417):
seid / seit ist nicht so schwer 🙂 Das kriegen selbst die Flüchtlinge hin!

anger scores: [0.500, 0.000]; 0.012 +/- 0.038
most angry comment (0.167):
Eine Schande das es die Tafel überhaupt gibt. Eine Schande das man sich nicht benehmen kann. Eine Schande......

fear scores: [0.250, 0.000]; 0.002 +/- 0.016
most afraid comment (0.083):
Die Frau hat doch bis jetzt nicht fertig gebracht ausser Zerstörung Deutschland

sad scores: [0.333, 0.000]; 0.005 +/- 0.024
most sad comment (0.118):
Das ist doch verdammt traurig!!!! Leider liest die merkel das sicher nicht!!!! Man empfindet nur noch wut!!!!!!!!!!!!!!!!!!

happy scores: [1.000, 0.000]; 0.065 +/- 0.158
most happy comment (1.000):
Wunderbar :)

##########
Kommentare/035 schweini papa.txt
sentiment scores: [1.000, -1.000]; 0.319 +/- 0.420
most positive comment (1.000):
Herzlichen Glückwunsch 🍼

most neg

sentiment scores: [1.000, -1.000]; 0.030 +/- 0.183
most positive comment (1.000):
Ja richtig! 😂

most negative comment (-0.500):
In dem Falle ergibt Minus & Minus aber kein Plus xD

anger scores: [1.000, 0.000]; 0.006 +/- 0.040
most angry comment (0.333):
Fra Nk besser still sein 🖕🖕🖕😂

fear scores: [0.500, 0.000]; 0.001 +/- 0.015
most afraid comment (0.364):
Weißt du nicht wer der Typ Ist?????😱😱😱😱

sad scores: [0.500, 0.000]; 0.002 +/- 0.023
most sad comment (0.222):
Aha Und uns haste schon wieder vergessen.....😭😛

happy scores: [1.000, 0.000]; 0.143 +/- 0.185
most happy comment (0.800):
Klar Schatzi 😘😘😂😂😂😂😂😂

##########
Kommentare/047 petition spahn.txt
sentiment scores: [1.000, -1.000]; -0.011 +/- 0.149
most positive comment (0.545):
Respekt Frau Grabau,sehr guter Beitrag ich wünsche Ihnen alles Gute

most negative comment (-0.429):
Blödsinn...keinen Monat...ein ganzes Jahr bitte...

anger scores: [1.000, 0.000]; 0.018 +/- 0.071
most angry comment (0.273):
Mindestens ein Jahr für das

...

sentiment scores: [1.000, -1.000]; -0.024 +/- 0.211
most positive comment (1.000):
Einfach glänzend!!!

most negative comment (-1.000):
Lächerlich 🤦🏻‍♂️😂😂😂😂

anger scores: [1.000, 0.000]; 0.035 +/- 0.114
most angry comment (0.500):
Möchtegern Deutscher...

fear scores: [0.167, 0.000]; 0.002 +/- 0.011
most afraid comment (0.067):
Was macht der in Bundestag der soll abhauen und die Claudia R gleich mit..Arrrrrrrrrrrr

sad scores: [0.333, 0.000]; 0.003 +/- 0.022
most sad comment (0.214):
Dann soll Cem die Artikel und Bücher von Yücel einmal lesen... 😣😣😣

happy scores: [1.000, 0.000]; 0.049 +/- 0.136
most happy comment (0.714):
Endlich mal....Glückwunsch 👍👍👍👍

##########
Kommentare/101 merkel tafel.txt
sentiment scores: [1.000, -0.500]; -0.018 +/- 0.139
most positive comment (0.667):
Super geschrieben! Klasse 👍😃

most negative comment (-0.500):
https://www.facebook.com/patrick.wietstock/videos/566859913669263/ BITTE TEILEN  ::::::::::::

anger scores: [1.000, 0.000]; 0.022 +/- 0.064
most a

sentiment scores: [1.000, -1.000]; 0.019 +/- 0.191
most positive comment (1.000):
Überlegung wert😅

most negative comment (-1.000):
Bloß nicht, 😂

anger scores: [0.667, 0.000]; 0.004 +/- 0.032
most angry comment (0.429):
Ahja 😂🖕🏼🖕🏼🖕🏼🖕🏼🖕🏼🖕🏼

fear scores: [0.500, 0.000]; 0.001 +/- 0.021
most afraid comment (0.500):
Never Meike Broer😳😳😳

sad scores: [0.500, 0.000]; 0.001 +/- 0.014
most sad comment (0.100):
Du hast dich selbst vergessen Thomas Alfter 😂😂😂

happy scores: [1.000, 0.000]; 0.151 +/- 0.208
most happy comment (1.000):
wunderschön 😂

##########
Kommentare/110 seehofer satz.txt
sentiment scores: [1.000, -1.000]; -0.004 +/- 0.171
most positive comment (0.500):
Er hat vollkommen Recht.

most negative comment (-0.500):
Dumm,dümmer, mitö kelz 🤦🏼‍♀️

anger scores: [1.000, 0.000]; 0.022 +/- 0.090
most angry comment (0.500):
Bayrischer Dummschwätzer

fear scores: [0.500, 0.000]; 0.003 +/- 0.025
most afraid comment (0.167):
Michael Kowalski einen Teufel   einen Teufel   einen Teufel   in s

sentiment scores: [1.000, -1.000]; -0.019 +/- 0.176
most positive comment (0.500):
Uiii neuer hashtagkrebs toll!

most negative comment (-1.000):
Vollidiot😂😂😂😂😂😂💯💯💯💯😂😂😂😂

anger scores: [1.000, 0.000]; 0.026 +/- 0.088
most angry comment (1.000):
Lächerlich, einfach lächerlich

fear scores: [0.500, 0.000]; 0.004 +/- 0.025
most afraid comment (0.200):
Stimmt , der muss Kurden töten ?

sad scores: [0.333, 0.000]; 0.003 +/- 0.015
most sad comment (0.091):
Aber leider leider ist das den Meisten hier nicht klar und die verstehen das auch absolut nicht. Leider. Trotzdem spielt sich hier jeder als Richter auf und der whataboutism ist allgegenwärtig. Richtig unsinnig!

happy scores: [1.000, 0.000]; 0.040 +/- 0.105
most happy comment (0.769):
Hahahahahahahaha omg xD 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂

##########
Kommentare/121 hussein k strafe.txt
sentiment scores: [1.000, -1.000]; -0.008 +/- 0.170
most positive comment (0.571):
Ein gerechtes Urteil. Viel Spaß im Knast

most negative comment (-0.500):
Ich bitte

sentiment scores: [1.000, -0.667]; 0.006 +/- 0.180
most positive comment (0.800):
Sehr guter Beitrag,der Hammer

most negative comment (-0.400):
Nein.  Und in beiden kriegen erlitten Sie eine schwere Niederlage. 😉

anger scores: [1.000, 0.000]; 0.020 +/- 0.084
most angry comment (0.500):
Und wieder Blabla Blabla....

fear scores: [0.125, 0.000]; 0.001 +/- 0.009
most afraid comment (0.032):
Der Islam gehört abgeschafft und in Deutschland hat der eh nichts zu suchen. ....wenn es denen nicht passt können Sie gerne wieder abhauen ihr Land und dort weiter im Mittelalter leben

sad scores: [0.250, 0.000]; 0.003 +/- 0.019
most sad comment (0.133):
Aber die AFD wurde dafür mimimi. 😢😢 Die haben Opferrolle schon derart verinnerlicht.😂

happy scores: [0.667, 0.000]; 0.036 +/- 0.085
most happy comment (0.429):
Andrea Schneidereit Quellenangabe ? Wikipedia ...🤣🤣🤣

##########
Kommentare/128 dobrinth ueber tafel.txt
sentiment scores: [1.000, -0.500]; 0.008 +/- 0.146
most positive comment (0.500):
All

sentiment scores: [1.000, -1.000]; -0.001 +/- 0.203
most positive comment (1.000):
Herzlichen Glückwunsch!!!

most negative comment (-0.500):
Tatsächlich Missbrauch?   https://www.google.de/amp/s/amp.n-tv.de/politik/Das-ist-erst-mal-keine-schlechte-Nachricht-article20347900.html

anger scores: [1.000, 0.000]; 0.032 +/- 0.125
most angry comment (0.750):
Unglaublich... Zum kotzen 😤

fear scores: [0.500, 0.000]; 0.002 +/- 0.027
most afraid comment (0.025):
Also, da muss Einem schon Angst und Bange werde, was unsere hochbezahlte, politische Elite mit der Vergabe unserer Steuergelder beschließt. Aber noch schlimmer ist, keine Partei ergreift die Initiative dies zu ändern. Das geht jetzt als so weiter und irgendwann schläft es ein oder wird vertagt. Hier gehts um Millionen Euro, die unserem Land gut tun würde und wir haben jetzt schon wieder in dieser Sache politischen Stillstand. Welcher Bundestagsabgeordnete garantiert mir, das diese Steuerverschwendung noch in dieser Periode korrigiert wi

## Visualization

In [4]:
# get all comments as lists of lists
sentences = []
for fname in comments_pp:
    for i in comments_pp[fname]:
        sentences.append(comments_pp[fname][i].split())

In [5]:
# train word2vec model
model = Word2Vec(sentences, size=50, window=10, min_count=3, workers=4)
word_vectors = model.wv
del model
word_vectors.init_sims(replace=True)

In [6]:
word_vectors.most_similar(positive=['🙈'])

[('🙊', 0.7755732536315918),
 ('😅', 0.7746701836585999),
 ('🙄', 0.6647796630859375),
 ('😜', 0.6192872524261475),
 ('😂', 0.6177088022232056),
 ('🤔', 0.5721880197525024),
 ('🙉', 0.5650612115859985),
 ('\U0001f92a', 0.5523953437805176),
 ('😏', 0.5502488017082214),
 ('😝', 0.5470948815345764)]

In [7]:
word_vectors['🙈']

array([-1.13752425e-01, -1.16185330e-01, -2.19351742e-02, -3.24434804e-04,
       -7.87170380e-02, -1.86275631e-01,  5.73057644e-02, -3.61884683e-02,
        1.02111928e-01,  1.53250039e-01,  1.77215915e-02,  1.65909171e-01,
       -3.62658985e-02,  9.06320289e-02, -5.59048541e-02,  2.89837420e-01,
       -2.90119573e-02,  3.47018540e-01, -3.38626951e-02,  5.12301438e-02,
       -5.30935489e-02, -3.03911343e-02,  1.33363113e-01, -1.09770164e-01,
        3.74665409e-02, -5.25749065e-02,  5.58801331e-02, -2.83183157e-01,
       -7.49809807e-03, -2.85950601e-01, -4.61526588e-02,  9.89458635e-02,
       -5.19315302e-02,  9.68127176e-02,  1.97869658e-01,  7.57222995e-02,
       -6.55743629e-02,  7.52899498e-02,  1.57749087e-01, -1.11482807e-01,
        2.03142688e-01,  2.53230006e-01,  2.24890769e-01,  2.54514724e-01,
        2.93506831e-02,  1.26056045e-01,  6.92617670e-02,  2.80804217e-01,
        6.17596209e-02,  1.18795177e-02], dtype=float32)

In [ ]:
# compute similarities between sentences
def sentence_sim(x, y):
    """
    cosine similarity between the average word embeddings of x and y (both lists of words)
    """
    vec_x = np.sum([word_vectors[word] for word in x if word in word_vectors], axis=0)
    vec_y = np.sum([word_vectors[word] for word in y if word in word_vectors], axis=0)
    if isinstance(vec_x, float) or isinstance(vec_y, float):
        return 0.  # no dot product with no vectors
    return np.dot(vec_x, vec_y) / (np.linalg.norm(vec_x) * np.linalg.norm(vec_y))

def sim_mat(fname):
    ids = sorted(comments_pp[fname])
    print("computing similarity matrix for %i comments" % len(ids))
    S = np.zeros((len(ids), len(ids)))
    for i in range(len(ids)):
        for j in range(i+1):
            similarity = sentence_sim(comments_pp[fname][ids[i]].split(), comments_pp[fname][ids[j]].split())
            if np.isnan(similarity):
                print(comments_pp[fname][ids[i]].split())
                print(comments_pp[fname][ids[j]].split())
            S[i, j], S[j, i] = similarity, similarity
    # scale to be between 0 and 1
    S -= S.min()
    S /= S.max()
    return S

In [ ]:
fname = fnames[1]
S = sim_mat(fname)
x, y = proj2d(S, use_tsne=True, verbose=True)
basic_viz(sorted(comments[fname]), emo_main_label[fname], x, y, dict(zip(emotions_list, emotions_list)), fname.split("/")[1].split(".")[0])